In [37]:
import torch
from torch import Tensor

In [38]:
target_seq = torch.tensor([1,3,2,4,0])

In [39]:
num_vertices = 10
vocab_size = 5

In [40]:
transition_matrix = torch.tensor(
    [
        [0, 0.9, 0.04, 0, 0.06, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0.5, 0.5, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    ]
)
emission_matrix = torch.ones(num_vertices, vocab_size) / vocab_size

In [41]:
def dfs(target_seq, transition_matrix, emission_matrix, seq, prob, storage):
    if len(seq) == len(target_seq):
        storage.append((seq, prob))
        return
    if len(seq) == 0:
        start_prob = emission_matrix[0][target_seq[0]]
        dfs(target_seq, transition_matrix, emission_matrix, [0], start_prob, storage)
    else:
        next_candidates = transition_matrix[seq[-1]]
        for i, p in enumerate(next_candidates):
            if p > 0:
                new_prob = prob * p * emission_matrix[i][target_seq[len(seq)]]
                dfs(target_seq, transition_matrix, emission_matrix, seq + [i], new_prob, storage)
                
def max_prob_path(storage):
    max_prob = 0
    max_seq = []
    for seq, prob in storage:
        if prob > max_prob:
            max_prob = prob
            max_seq = seq
    return max_seq, max_prob

In [42]:
storage = []

In [43]:
dfs(target_seq, transition_matrix, emission_matrix, [], 1, storage)

In [44]:
total_prob = sum([prob for seq, prob in storage])

In [45]:
max_prob_path(storage)

([0, 1, 6, 7, 9], tensor(0.0003))

In [46]:
def dfs_lynchpin(target_seq, transition_matrix, emission_matrix, seq, prob, storage, assignments=None):
    if len(seq) == len(target_seq):
        storage.append((seq, prob))
        return
    if len(seq) == 0:
        start_prob = emission_matrix[0][target_seq[0]]
        dfs_lynchpin(target_seq, transition_matrix, emission_matrix, [0], start_prob, storage, assignments)
    else:
        next_candidates = transition_matrix[seq[-1]]
        current_assignment = -1 if assignments is None else assignments[len(seq)]
        for i, p in enumerate(next_candidates):
            if p > 0 and (current_assignment == -1 or current_assignment == i):
                new_prob = prob * p * emission_matrix[i][target_seq[len(seq)]]
                dfs_lynchpin(
                    target_seq, 
                    transition_matrix, 
                    emission_matrix, 
                    seq + [i], 
                    new_prob, 
                    storage,
                    assignments)

In [47]:
storage = []

In [48]:
assignments = torch.tensor([-1, 4, -1, 7, -1])

In [49]:
dfs_lynchpin(target_seq, transition_matrix, emission_matrix, [], 1, storage, assignments)

In [50]:
storage

[([0, 4, 5, 7, 9], tensor(9.6000e-06)), ([0, 4, 6, 7, 9], tensor(9.6000e-06))]

In [51]:
total_lynchpin_prob = sum([prob for seq, prob in storage])

In [52]:
import einops

In [53]:
def vector_gather(vectors, indices):
    """
    Gathers (batched) vectors according to indices.
    Arguments:
        vectors: Tensor[N, L, D]
        indices: Tensor[N, K] or Tensor[N]
    Returns:
        Tensor[N, K, D] or Tensor[N, D]
    """
    N, L, D = vectors.shape
    squeeze = False
    if indices.ndim == 1:
        squeeze = True
        indices = indices.unsqueeze(-1)
    N2, K = indices.shape
    assert N == N2
    indices = einops.repeat(indices, "N K -> N K D", D=D)
    out = torch.gather(vectors, dim=1, index=indices)
    if squeeze:
        out = out.squeeze(1)
    return out

In [54]:
def logsumexp(x: Tensor, dim: int) -> Tensor:
    # Solving nan issue when x contains -inf
    # See https://github.com/pytorch/pytorch/issues/31829
    # https://github.com/thu-coai/DA-Transformer/blob/main/fs_plugins/custom_ops/dag_loss.py
    m, _ = x.max(dim=dim, keepdim=True)
    mask = m == -float("inf")
    m = m.detach()
    s = (x - m.masked_fill_(mask, 0)).exp_().sum(dim=dim, keepdim=True)
    return s.masked_fill_(mask, 1).log_() + m.masked_fill_(mask, -float("inf"))

In [55]:
def dag_loss_raw(targets, transition_matrix, emission_probs):
    """
    Calculates the directed acyclic graph (DAG) loss given the targets, transition matrix, and emission probabilities.
    It returns the dynamic programming table of which one of the entries is the DAG loss.

    Args:
        targets (torch.Tensor): The target sequence of shape (batch_size, m).
        transition_matrix (torch.Tensor): The transition matrix of shape (batch_size, l, l).
        emission_probs (torch.Tensor): The emission probabilities of shape (batch_size, l, vocab_size).

    Returns:
        torch.Tensor: The DAG loss of shape (batch_size, m, l).
    """
    batch_size, m = targets.shape
    _, l, vocab_size = emission_probs.shape
    dp = torch.ones((batch_size, m, l), device=transition_matrix.device)
    dp[dp == 1] = -float("inf")
    initial_probs = torch.gather(
        emission_probs, dim=2, index=targets[:, 0].unsqueeze(1).unsqueeze(2)
    )
    dp[:, 0, 0] = initial_probs.squeeze(2).squeeze(1)
    # assumes that transition_matrix and emission_probs are already in log space
    # also we need to tranpose emission_probs so it is vocab_size x l
    # so the vector gather works
    emission_probs = emission_probs.transpose(1, 2)
    for i in range(1, m):
        dp[:, i, :] = vector_gather(emission_probs, targets[:, i]) + (
            (
                logsumexp(
                    dp[:, i - 1, :].unsqueeze(1).transpose(1, 2) + transition_matrix,
                    dim=1,
                )
            ).squeeze(1)
        )
    return dp

In [56]:
transition_matrix = torch.log(transition_matrix)

In [57]:
emission_matrix = torch.log(emission_matrix)

In [58]:
transition_matrix = transition_matrix.unsqueeze(0)
emission_matrix = emission_matrix.unsqueeze(0)

In [59]:
target_seq = target_seq.unsqueeze(0)

In [60]:
dp = dag_loss_raw(target_seq, transition_matrix, emission_matrix)

In [61]:
torch.exp(dp)

tensor([[[2.0000e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 3.6000e-02, 1.6000e-03, 0.0000e+00, 2.4000e-03,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 3.2000e-04, 0.0000e+00,
          2.4000e-04, 7.4400e-03, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 1.5360e-03, 6.4000e-05, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 3.2000e-04]]])

In [62]:
float(total_prob)

0.00032000005012378097

In [63]:
float(torch.exp(dp[0][-1][-1]))

0.0003199999628122896

In [64]:
def correct_dp_slice_for_spans(
    dp_slice: Tensor, target_span_indices: Tensor, iteration: int
) -> Tensor:
    """
    @param dp: (batch_size, num_vertices, 1) tensor of scores that is being consiered
        by the current iteration (position in target sequence)
    @param target_span_indices: (batch_size, seq_len) tensor of indices, if element
        is non-negative, it represents that position in target sequence span otherwise
        it is negative
    @param iteration: int, the current iteration in the target sequence

    @return dp_slice: (batch_size, num_vertices, 1) tensor of scores that guarantees
        that all paths considered go through the vertex specified by the target_span_indices
        if it is non-negative, otherwise scores of paths are not changed to -inf
    """
    res = dp_slice
    res = res.squeeze(2)

    # here we cannot simply use this res to update the dp table,
    # because this res update assumes that all paths are allowed,
    # however, if we are at a span, only the path that passes
    # through the span's specified vertex is allowed
    # to do this, we set the probability of all other vertices
    # to -inf, so that they are not considered in the max operation
    # in the next step. This guarantees in the next step, we only
    # consider paths that pass through the span's specified vertex
    curr_span_status = target_span_indices[:, iteration - 1]
    in_span = curr_span_status >= 0

    # we need to do a masked_fill here for curr_span_status, because
    # we use gather to get the current probability of the span vertex
    # in res, and gather doesn't support negative indices.
    # this also means the corresponding probability might not be useful
    # because if it is not a span then we don't enforce that paths
    # must pass through the span's vertex. We account for this later
    curr_span_status = curr_span_status.masked_fill(~in_span, 0).unsqueeze(1)

    selection = res.gather(1, curr_span_status)

    masked = torch.full_like(res, -float("inf"))
    masked.scatter_(1, curr_span_status, selection)

    in_span = in_span.unsqueeze(-1)
    in_span = in_span.expand_as(res)

    res = torch.where(in_span, masked, res)
    return res

In [65]:
def dag_loss_raw_lynchpin(targets, transition_matrix, emission_probs, assignments):
    """
    Calculates the directed acyclic graph (DAG) loss given the targets, transition matrix, and emission probabilities.
    It returns the dynamic programming table of which one of the entries is the DAG loss.

    Args:
        targets (torch.Tensor): The target sequence of shape (batch_size, m).
        transition_matrix (torch.Tensor): The transition matrix of shape (batch_size, l, l).
        emission_probs (torch.Tensor): The emission probabilities of shape (batch_size, l, vocab_size).
        assignments (torch.Tensor): The assignments of shape (batch_size, m).

    Returns:
        torch.Tensor: The DAG loss of shape (batch_size, m, l).
        
    The assignments tensor is a tensor of shape (batch_size, m) where each element is an integer from 0 to l-1.
    If the element is -1, then the corresponding element in the target sequence is free to be any of the l vertices.
    Otherwise, at that position in the target sequence, the vertex must be the one specified by the assignment.
    """
    batch_size, m = targets.shape
    _, l, vocab_size = emission_probs.shape
    dp = torch.ones((batch_size, m, l), device=transition_matrix.device)
    dp[dp == 1] = -float("inf")
    initial_probs = torch.gather(
        emission_probs, dim=2, index=targets[:, 0].unsqueeze(1).unsqueeze(2)
    )
    dp[:, 0, 0] = initial_probs.squeeze(2).squeeze(1)
    # assumes that transition_matrix and emission_probs are already in log space
    # also we need to tranpose emission_probs so it is vocab_size x l
    # so the vector gather works
    emission_probs = emission_probs.transpose(1, 2)
    for i in range(1, m):
        # before proceeding, we need to check the previous iteration's dp values
        # to see if that it agrees with the assignments, if not, we need to adjust
        # the dp values accordingly
        prev_dp = dp[:, i - 1, :].unsqueeze(1).transpose(-1, -2)
        prev_dp = correct_dp_slice_for_spans(prev_dp, assignments, i)
        dp[:, i - 1, :] = prev_dp
        dp[:, i, :] = vector_gather(emission_probs, targets[:, i]) + (
            (
                logsumexp(
                    dp[:, i - 1, :].unsqueeze(1).transpose(1, 2) + transition_matrix,
                    dim=1,
                )
            ).squeeze(1)
        )
    return dp

In [66]:
assignments = assignments.unsqueeze(0)

In [67]:
dp2 = dag_loss_raw_lynchpin(target_seq, transition_matrix, emission_matrix, assignments)

In [68]:
dp2

tensor([[[ -1.6094,     -inf,     -inf,     -inf,     -inf,     -inf,     -inf,
              -inf,     -inf,     -inf],
         [    -inf,     -inf,     -inf,     -inf,  -6.0323,     -inf,     -inf,
              -inf,     -inf,     -inf],
         [    -inf,     -inf,     -inf,     -inf,     -inf,  -8.3349,  -8.3349,
              -inf,     -inf,     -inf],
         [    -inf,     -inf,     -inf,     -inf,     -inf,     -inf,     -inf,
           -9.2512,     -inf,     -inf],
         [    -inf,     -inf,     -inf,     -inf,     -inf,     -inf,     -inf,
              -inf,     -inf, -10.8606]]])

In [69]:
float(total_lynchpin_prob)

1.920000067912042e-05

In [70]:
float(torch.exp(dp2[0][-1][-1]))

1.920001523103565e-05

In [71]:
from typing import Tuple

In [72]:
def findPathBatched(
    transition_matrix: Tensor,
    emission_probs: Tensor,
    target_seq: Tensor,
    target_span_indices: Tensor,
) -> Tuple[Tensor, Tensor]:
    """
    @param transition_matrix: (b, n, n) tensor, `n` is number of vertices
        the matrix should be in log-space, and describe an acyclic directed graph
    @param emission_probs: (b, n, m) tensor, where `n` is the number of vertices
        and `m` is the vocabulary size. The matrix should be in log-space
    @param target_seq: (b, seq_len) tensor
    @param target_span_indices: (b, seq_len) tensor, the indices of the target span
        for their respective elements in `target_seq`. If the element in `target_seq`
        is a span, then the corresponding element in `target_span_indices` should be
        non-negative, otherwise it should be negative.

    @return: (dp, backtrace) where `dp` is a (b, seq_len, n) tensor and `backtrace`
        is a (b, seq_len - 1, n) tensor. `dp` contains the log-probabilities of the
        most likely path to each vertex at each time step. `backtrace` contains
        the backtrace pointers for each vertex at each time step.

    Find the most likely path through the graph given the target sequence and
    the transition and emission probabilities. The graph is an acyclic directed
    graph, and the transition and emission probabilities are in log-space.

    The path must start at the first vertex and end at the last vertex with
    the number of vertices in the path equal to the length of the target sequence.
    """
    b, n, m = emission_probs.shape
    _, seq_len = target_seq.shape
    dp = torch.zeros(b, seq_len, n, device=transition_matrix.device)
    dp.fill_(-float("inf"))
    initial_probs = torch.gather(
        emission_probs, dim=2, index=target_seq[:, 0].unsqueeze(1).unsqueeze(2)
    )
    dp[:, 0, 0] = initial_probs.squeeze(2).squeeze(1)
    backtrace = torch.zeros(
        b, seq_len - 1, n, dtype=torch.long, device=transition_matrix.device
    )

    emission_probs = emission_probs.transpose(-1, -2)
    for i in range(1, seq_len):
        currDp = dp[:, i - 1, :].unsqueeze(1).transpose(-1, -2)
        currDp = correct_dp_slice_for_spans(currDp, target_span_indices, i)
        dp[:, i - 1, :] = currDp
        currDp = currDp.unsqueeze(2)
        pathProbs = currDp + transition_matrix
        max_v, max_idx = torch.max(pathProbs, dim=1)
        res = max_v + vector_gather(emission_probs, target_seq[:, i])
        dp[:, i, :] = res
        backtrace[:, i - 1, :] = max_idx
    return dp, backtrace


def backtrace(
    backtrace: Tensor,
    target_lens: Tensor,
    vertex_lens: Tensor,
) -> Tuple[Tensor, Tensor]:
    """
    @param backtrace: (batch_size, seq_len - 1, n) tensor
    @param target_lens: (batch_size,) tensor
    @param vertex_lens: (batch_size,) tensor
    @param target_span_indices: (batch_size, seq_len) tensor
    @param target_span_indices_pad_value: int

    @return: (paths, is_padding) where `paths` is a (batch_size, seq_len) tensor
        and `is_padding` is a (batch_size, seq_len) tensor. `paths` contains the
        most likely path through the graph given the backtrace tensor. `is_padding`
        is a boolean tensor that indicates whether the path is padding or not.
        If the respective element in `is_padding` is True, then the path at that
        position is padding.

    Given the backtrace tensor, find the most likely path
    that starts from the first vertex and ends at the last vertex.

    For any padding vertices, we assume that the previous vertex
    is where the path comes from
    """
    b, t, v = backtrace.shape
    offset_target_lens = target_lens - 1
    start_vertex = vertex_lens - 1
    backtrace = backtrace.transpose(-1, -2)
    tracepath = torch.zeros(b, dtype=torch.long, device=backtrace.device) + v - 1
    traces = []
    for i in range(t - 1, -1, -1):
        tracepath = torch.where((i + 1) == offset_target_lens, start_vertex, tracepath)
        nextpath = vector_gather(backtrace, tracepath)[:, i]
        nextpath = torch.where(nextpath == 0, tracepath - 1, nextpath)
        tracepath = torch.where(i >= offset_target_lens, -1, tracepath)
        traces.append(tracepath)
        tracepath = nextpath
    traces.append(tracepath * 0)
    traces = torch.stack(traces[::-1], dim=-1)
    return traces, traces < 0

In [73]:
dp_path, dp_bt = findPathBatched(transition_matrix, emission_matrix, target_seq, assignments)

In [77]:
traces, is_padding = backtrace(dp_bt, torch.tensor([5]), torch.tensor([10]))

In [78]:
traces

tensor([[0, 4, 5, 7, 9]])